In [29]:
import numpy as np
import pandas as pd
import datasets

import gadgets

In [30]:
orig_aqua = datasets.load_dataset("aqua_rat")

In [31]:
orig_aqua["train"][0]

{'question': "Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other?",
 'options': ['A)21', 'B)21.5', 'C)22', 'D)22.5', 'E)23'],
 'rationale': 'If Q complete x kilometers, then P completes 1.15x kilometers.\nx + 1.15x = 43\n2.15x=43\nx = 43/2.15 = 20\nThen P will have have walked 1.15*20=23 km.\nThe answer is E.',
 'correct': 'E'}

In [32]:
aqua = {}

def parse_options(options: list[str]) -> dict[str, str]:
    assert len(options) == 5
    options_dict = {"A": None, "B": None, "C": None, "D": None, "E": None}
    for option, letter in zip(options, options_dict.keys()):
        prefix = letter + ")"
        assert option.startswith(prefix)
        options_dict[letter] = option.removeprefix(prefix).strip()
    return options_dict

for split in orig_aqua.keys():
    aqua[split] = orig_aqua[split].to_pandas()
    aqua[split]["options"] = aqua[split]["options"].apply(parse_options)
    aqua[split]["correct"] = aqua[split]["correct"].str.upper().str.strip()
    assert set(aqua[split]["correct"].unique()) == {"A", "B", "C", "D", "E"}


In [33]:
import pandarallel

pandarallel.pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [34]:
def transform_chain(row) -> str:
    example = gadgets.aqua.parse(row.to_dict())
    markup = gadgets.markup.to_model_markup(example=example)
    return str(markup)

for split in orig_aqua.keys():
    aqua[split]["chain"] = orig_aqua[split].to_pandas().parallel_apply(transform_chain, axis=1)

/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.Beautifu

/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.Beautifu

/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.Beautifu

In [35]:
aqua["train"].head()
for split in aqua.keys():
    aqua[split] = pd.DataFrame({
        "question": aqua[split]["question"],
        "options": aqua[split]["options"],
        "chain": aqua[split]["chain"],
        "result": aqua[split]["correct"],
    })
    

In [36]:
aqua["train"].head()

,question,options,chain,result
0,"Two friends plan to walk along a 43-km trail, ...","{'A': '21', 'B': '21.5', 'C': '22', 'D': '22.5...","If Q complete x kilometers, then P completes 1...",E
1,"In the coordinate plane, points (x, 1) and (5,...","{'A': '4 and 1', 'B': '1 and 5', 'C': '5 and 1...",Line k passes through the origin and has slope...,C
2,"For all numbers p and q, the operation @ is de...","{'A': 'II', 'B': 'I and II', 'C': 'I and III',...",p@q = p^2 - pq=p(p-q).... so p@q will be zero ...,B
3,Carl is facing very difficult financial times ...,"{'A': '$1600', 'B': '$2000', 'C': '$2150', 'D'...","Usually, you are given the annual rate of inte...",A
4,The speed at which a man can row a boat in sti...,"{'A': '18 seconds', 'B': '27 seconds', 'C': '2...",Speed of the boat downstream = 25 +11= \n<gadg...,E


In [39]:
# datasets.DatasetDict({
#     "train": datasets.Dataset.from_pandas(aqua["train"]),
#     "validation": datasets.Dataset.from_pandas(aqua["validation"]),
#     "test": datasets.Dataset.from_pandas(aqua["test"]),
# }).push_to_hub("MU-NLPC/Calc-aqua_rat")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/98 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]